In [ ]:
pip install streamlit scikit-learn pandas numpy matplotlib

In [2]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [3]:
# -------------------------------
# Simulated Data (mtcars equivalent)
# -------------------------------
from sklearn.datasets import load_iris
iris = load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)

# For demo purposes, rename columns to look like financial indicators
data.columns = ["income", "debt", "savings", "assets"]

# Create financial clusters
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data[["income", "debt", "savings", "assets"]])
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
data["cluster"] = kmeans.fit_predict(X_scaled)

# Simulate education as binary: 1 = College+, 0 = No College
np.random.seed(42)
data["education"] = np.where(data["cluster"] == 2, 1, np.random.binomial(1, 0.5, len(data)))

# Fit logistic regression: education ~ cluster + financial features
X = pd.get_dummies(data[["cluster", "income", "debt", "savings", "assets"]], drop_first=True)
y = data["education"]
logit = LogisticRegression()
logit.fit(X, y)

LogisticRegression()

In [4]:
# -------------------------------
# Streamlit App
# -------------------------------
st.title("🎓 Build Your Financial Profile")
st.markdown("""
Enter your household’s approximate financial information below to see which **financial cluster** you most closely resemble — and the estimated likelihood of holding a college degree or higher.
""")

# User inputs
income = st.slider("Annual Income ($)", 20000, 200000, 60000, 5000)
debt = st.slider("Total Debt ($)", 0, 200000, 50000, 5000)
savings = st.slider("Liquid Savings ($)", 0, 100000, 20000, 2000)
assets = st.slider("Total Assets ($)", 0, 500000, 100000, 10000)

# Predict cluster
user_input = pd.DataFrame([[income, debt, savings, assets]], columns=["income", "debt", "savings", "assets"])
user_scaled = scaler.transform(user_input)
user_cluster = kmeans.predict(user_scaled)[0]

# Predict probability of higher education
X_user = pd.DataFrame({
    "cluster": [user_cluster],
    "income": [income],
    "debt": [debt],
    "savings": [savings],
    "assets": [assets]
})
X_user = pd.get_dummies(X_user, columns=["cluster"], drop_first=True).reindex(columns=X.columns, fill_value=0)
prob = logit.predict_proba(X_user)[0][1]

# -------------------------------
# Display results
# -------------------------------
st.subheader("🔍 Your Results")
st.write(f"**Your profile most closely matches Cluster {user_cluster + 1}.**")
st.write(f"Estimated probability of holding a college degree: **{prob*100:.1f}%**")

# Visualize where user falls in cluster space
fig, ax = plt.subplots()
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=data["cluster"], cmap="viridis", alpha=0.5)
plt.scatter(user_scaled[:, 0], user_scaled[:, 1], color="red", s=100, marker="^", label="You")
plt.xlabel("Scaled Income / Debt Dimension")
plt.ylabel("Scaled Savings / Assets Dimension")
plt.title("Financial Clusters Visualization")
plt.legend()
st.pyplot(fig)

st.markdown("""
**Interpretation Example:**
- Cluster 1 → Low-wealth, low-debt households  
- Cluster 2 → Middle-income, high-debt households  
- Cluster 3 → High-asset, low-leverage households  
""")

2025-11-10 10:00:26.212 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 10:00:27.217 
  command:

    streamlit run C:\Users\Samantha\anaconda_work\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-10 10:00:27.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 10:00:27.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 10:00:27.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 10:00:27.222 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 10:00:27.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 10:00:27.224 Thread 'MainThre

DeltaGenerator()

In [5]:
streamlit run financial_profile_app.py

SyntaxError: invalid syntax (857880790.py, line 1)

In [6]:
!pip install gradio


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
policyengine-core 3.8.1 requires numpy~=1.26.4, but you have numpy 2.0.2 which is incompatible.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.5.0 which is incompatible.



  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.12.2
    Uninstalling typing-extensions-4.12.2:
      Successfully uninstalled typing-extensions-4.12.2
  Attempting uninstall: anyio
    Found existing installation: anyio 3.5.0
    Uninstalling anyio-3.5.0:
      Successfully uninstalled anyio-3.5.0


In [7]:
import gradio as gr
import pandas as pd

def classify_profile(education, income, debt):
    if income > 100000 and debt > 50000:
        cluster = "High-income, high-debt"
    elif income < 50000 and debt < 20000:
        cluster = "Low-income, low-debt"
    else:
        cluster = "Moderate-income, renters"
    return f"You match Cluster: {cluster}"

demo = gr.Interface(
    fn=classify_profile,
    inputs=[
        gr.Radio(["High school", "Bachelor's", "Graduate"], label="Education"),
        gr.Slider(0, 200000, label="Income ($)"),
        gr.Slider(0, 100000, label="Debt ($)")
    ],
    outputs="text",
    title="Build Your Financial Profile"
)

demo.launch()


ImportError: cannot import name 'Sentinel' from 'typing_extensions' (C:\Users\Samantha\anaconda_work\lib\site-packages\typing_extensions.py)